Keras Tuner는 TensorFlow 프로그램에 대한 최적의 하이퍼파라미터 세트를 선택하는 데 도움을 주는 라이브러리입니다.

# 준비

## 라이브러리 호출

In [2]:
import tensorflow as tf
from tensorflow import keras
import IPython
import kerastuner as kt

C:\Users\user\miniconda3\envs\deep_tf\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  after removing the cwd from sys.path.


## 데이터셋 로드

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
# Normalize pixel values between 0 and 1
# dtype='uint8' -> dtype='float32'

img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# 하이퍼 튜닝

## 하이퍼 튜닝을 적용할 모델 정의

튜닝을 적용할 objective 설정

In [8]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Dense 층 unit 의 개수를 32~512 까지 32개의 step 으로 나누어 입력
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    # learning_rate를 0.01, 0.001, 0.0001 중 선택하여 입력
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['acc'])

    return model

## 튜너를 정의

keras의 Tuner 에는 `RandomSearch`, `HyperBand`, `BayesianOptimization`, `Sklearn` 이렇게 4가지 튜너가 있습니다.

In [13]:
tuner = kt.Hyperband(model_builder,
                    objective= "val_acc",
                     max_epochs= 10,
                     factor = 3,
                     directory = "my_dir",
                     project_name = "intro_to_kt")

Hyperband 튜닝 알고리즘은 적응형 리소스 할당 및 조기 중단을 사용하여 고성능 모델에서 신속하게 수렴합니다. 

이 알고리즘은 몇 개의 epoch에 대해 많은 수의 모델을 훈련하고 최고 성능을 보이는 절반만 다음 단계로 넘깁니다.

## 콜백 정의

하이퍼파라미터 검색을 실행하기 전에 훈련 단계가 끝날 때마다 훈련 결과를 지우도록 콜백을 정의합니다.

In [10]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):

    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

## 하이퍼 파라미터 탐색

In [14]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 11s]
val_acc: 0.8844000101089478

Best val_acc So Far: 0.8866000175476074
Total elapsed time: 00h 03m 07s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.



Dense layer의 unit 이 416 일때, learning_rate 가 0.001 일때 가장 좋은 성능을 낸다는 것을 확인 할 수 있습니다.

# 최적의 하이퍼파라미터로 모델 훈련

In [15]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4788 - acc: 0.8297 - val_loss: 0.4077 - val_acc: 0.8572
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3568 - acc: 0.8709 - val_loss: 0.3724 - val_acc: 0.8653
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3213 - acc: 0.8828 - val_loss: 0.3553 - val_acc: 0.8700
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2984 - acc: 0.8899 - val_loss: 0.3476 - val_acc: 0.8746
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2790 - acc: 0.8964 - val_loss: 0.3558 - val_acc: 0.8759
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2661 - acc: 0.9010 - val_loss: 0.3409 - val_acc: 0.8800
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2530 - acc: 0.9054 - val_loss: 0.3230 - val_acc: 0.8846
Epoch 8/10
1875/1875 [==============================] - 2s 1ms